# SimSim

In [1]:
import json
import math
import networkx as nx
import community
from collections import defaultdict, Counter
from ipysigma import Sigma
from fog.metrics import sparse_dot_product
from tqdm import tqdm_notebook

## Loading the recipes

In [2]:
with open('./recipes.json', 'r') as f:
    RECIPES = json.load(f)

for i, recipe in enumerate(RECIPES[0:2]):
    title = 'Recipe n°%i' % (i + 1)
    print(title)
    print('-' * len(title))
    
    for ingredient in recipe['ingredients']:
        print(ingredient)
        
    print()
print('...')

Recipe n°1
----------
romaine lettuce
black olives
grape tomatoes
garlic
pepper
purple onion
seasoning
garbanzo beans
feta cheese crumbles

Recipe n°2
----------
plain flour
ground pepper
salt
tomatoes
ground black pepper
thyme
eggs
green tomatoes
yellow corn meal
milk
vegetable oil

...


## Computing the similarity matrix

In [3]:
VECTORS = defaultdict(Counter)
OCCURRENCES = Counter()
for recipe in RECIPES:
    ingredients = recipe['ingredients']
    
    for i in range(len(ingredients)):
        A = ingredients[i]
        OCCURRENCES[A] += 1
        for j in range(i + 1, len(ingredients)):
            B = ingredients[j]
            
            VECTORS[A][B] += 1
            VECTORS[B][A] += 1
            
print(VECTORS['milk'].most_common(5))
print(OCCURRENCES['milk'])
print(len(VECTORS))

[('salt', 1354), ('butter', 835), ('all-purpose flour', 791), ('eggs', 734), ('sugar', 598)]
2263
6714


In [4]:
# Trimming
FREQ_THRESHOLD = 10

TO_DROP = set()

for i, c in OCCURRENCES.items():
    if c < FREQ_THRESHOLD:
        TO_DROP.add(i)
        
for i in TO_DROP:
    del VECTORS[i]
    
for i, v in VECTORS.items():
    for j in TO_DROP:
        if j in v:
            del v[j]

In [5]:
NORMS = {}
SUMS = {}
for ingredient, vector in VECTORS.items():
    NORMS[ingredient] = math.sqrt(sum(map(lambda x: x * x, vector.values())))
    SUMS[ingredient] = sum(vector.values())
    
NORMS['milk']

2372.090428293154

In [6]:
# PMI computations
PMI = defaultdict(dict)
N_TOT = sum(sum(x for x in v.values()) for v in VECTORS.values()) / 2

for kA, A in tqdm_notebook(VECTORS.items()):
    nA = NORMS[kA]
    for kB, nab in A.items():
        B = VECTORS[kB]
        nB = NORMS[kB]
        # s = sparse_dot_product(A, B) / (nA * nB)
        s = math.log(nab * N_TOT / (SUMS[kA] * SUMS[kB]))
    
        if s > 1:
            PMI[kA][kB] = s
            PMI[kB][kA] = s

In [7]:
MATRIX = defaultdict(dict)

for kA, A in tqdm_notebook(VECTORS.items()):
    nA = NORMS[kA]
    
    sA = sum(PMI[kA].values())
    
    for kB in A:
        B = VECTORS[kB]
        nB = NORMS[kB]
        
        ks = set(PMI[kA].keys()) & set(PMI[kB].keys())
        
        d = sum(min(PMI[kA][k], PMI[kB][k]) for k in ks)
            
        if sA <= 0:
            continue
        
        s = d / sA

        MATRIX[kA][kB] = s
        MATRIX[kB][kA] = s

In [8]:
# Test
MATRIX['milk']

{'romaine lettuce': 0.010689893799720046,
 'black olives': 0.009920202011598885,
 'grape tomatoes': 0.035200993180384804,
 'pepper': 0.0,
 'purple onion': 0.009940609088630868,
 'seasoning': 0.018862597448802604,
 'garbanzo beans': 0.020470886631503423,
 'feta cheese crumbles': 0.0,
 'plain flour': 0.15878446278195724,
 'ground pepper': 0.018072156829853518,
 'tomatoes': 0.012116766586827275,
 'thyme': 0.012441126191313111,
 'eggs': 0.20021385263931726,
 'green tomatoes': 0.04131111262329912,
 'yellow corn meal': 0.10741547432302684,
 'salt': 0.0,
 'ground black pepper': 0.0,
 'vegetable oil': 0.0,
 'black pepper': 0.0,
 'shallots': 0.0,
 'cornflour': 0.05193249702476836,
 'cayenne pepper': 0.0,
 'onions': 0.0,
 'garlic paste': 0.01984117885882317,
 'butter': 0.43169513417798383,
 'lemon juice': 0.03129286280260807,
 'water': 0.0,
 'chili powder': 0.0,
 'passata': 0.007568403437154561,
 'oil': 0.0157316747337883,
 'ground cumin': 0.0,
 'boneless chicken skinless thigh': 0.0,
 'garam ma

## Computing the macro similarity graph

In [9]:
# Starting threshold
STARTING_THRESHOLD = 0.25

# Iwanthue palette for ten first communities
PALETTE = [
    '#c9587a',
    '#5dba5a',
    '#b65cbf',
    '#a3af44',
    '#747bc8',
    '#d49d43',
    '#4fbab7',
    '#d35238',
    '#588042',
    '#ac6f40'
]

In [10]:
similarity_graph = nx.Graph()

for ingredient, neighbors in MATRIX.items():
    for neighbor, cosine in neighbors.items():
        if cosine >= STARTING_THRESHOLD:
            similarity_graph.add_edge(ingredient, neighbor, cosine=cosine)

len(similarity_graph)

1255

In [11]:
# Keeping only largest component
largest_component = set(max(nx.connected_components(similarity_graph), key=len))

for node in list(similarity_graph.nodes):
    if node not in largest_component:
        similarity_graph.remove_node(node)

In [12]:
# Community detection using the Louvain method
partition = community.best_partition(similarity_graph)

# Counting
communities_count = Counter()

for node, c in partition.items():
    communities_count[c] += 1
    
communities_colors = {}

i = 0
for c, _ in communities_count.most_common(10):
    communities_colors[c] = PALETTE[i]
    i += 1

In [118]:
# Styling
for node, attrs in similarity_graph.nodes(data=True):
    attrs['size'] = min(12, max(2, OCCURRENCES[node] / len(OCCURRENCES) * 6))
    attrs['color'] = communities_colors.get(partition[node], '#BBB')

# Display
Sigma(similarity_graph, height=1000)

Sigma(data={'nodes': [('romaine lettuce', {'size': 2, 'color': '#ac6f40'}), ('black pepper', {'size': 2.347631…

## Computing the micro similarity graphs

In [58]:
communities = [set() for i in range(len(communities_count))]

for node, community in partition.items():
    communities[community].add(node)
    
communities = sorted(communities, key=lambda x: -len(x))

In [59]:
SUBVECTORS = {}

for community in communities:
    for item in community:
        SUBVECTORS[item] = Counter(dict((key, count) for key, count in VECTORS[item].items() if key in community))
        
SUBVECTORS['milk']

Counter({'eggs': 734, 'butter': 835, 'flour': 308})

In [70]:
SUBNORMS = {}
SUBSUMS = {}
for ingredient, vector in SUBVECTORS.items():
    SUBNORMS[ingredient] = math.sqrt(sum(map(lambda x: x * x, vector.values())))
    SUBSUMS[ingredient] = sum(vector.values())
    
SUBNORMS['milk']

1153.6225552580013

In [61]:
# PMI computations
PMIS = []

for community in communities:
    pmi = defaultdict(dict)
    ntot = sum(sum(x for x in v.values()) for v in SUBVECTORS.values()) / 2
    PMIS.append(pmi)
    
    community = list(community)

    for i, kA in enumerate(community):
        for j in range(i + 1, len(community)):
            kB = community[j]
            nab = SUBVECTORS[kA].get(kB)
            
            if not nab:
                continue
            
            s = math.log(nab * ntot / (SUBSUMS[kA] * SUBSUMS[kB]))

            if s > 1:
                pmi[kA][kB] = s
                pmi[kB][kA] = s

In [98]:
MATRICES = []

for i, community in enumerate(communities):
    pmi = PMIS[i]
    m = defaultdict(dict)
    MATRICES.append(m)
    
    community = list(community)

    for i, kA in enumerate(community):
        sA = sum(pmi[kA].values())
        
        for j in range(i + 1, len(community)):
            kB = community[j]
            
            ks = set(pmi[kA].keys()) & set(pmi[kB].keys())

            d = sum(min(pmi[kA][k], pmi[kB][k]) for k in ks)

            if sA <= 0:
                continue

            s = d / sA

            m[kA][kB] = s
            m[kB][kA] = s

In [99]:
for i, c in enumerate(communities):
    if 'dried tart cherries' in c:
        for j, k in sorted(PMIS[i]['dried tart cherries'].items()):
            print(j, k, PMIS[i]['candied fruit'].get(j))

almond extract 2.131135156328938 None
almonds 1.6403283994534388 None
bartlett pears 3.662611527293326 None
candied orange peel 2.6817822742815998 None
caramel sauce 3.496626389819065 None
cherries 2.7574940960172962 2.7679653958845916
chocolate 2.7687936512712295 None
chopped walnuts 1.7530690224088876 None
cream of tartar 1.1469332188385721 None
crème fraîche 2.1585341305170522 None
dried apricot 1.9378627683482312 None
egg whites 1.406943354087032 1.4174146539543275
extract 3.7803945629497098 None
golden brown sugar 1.9428255576903604 None
golden raisins 2.0407510948606684 2.4566875028361284
grated lemon peel 1.7286775692847285 None
grated lemon zest 1.2459539780394326 1.256425277906728
grated orange 1.7186874762096422 1.7291587760769376
grated orange peel 2.536600264437102 1.8539243837444521
ground allspice 1.0285268583474458 None
ground almonds 2.6359727382503055 2.646444038117601
hazelnuts 2.5187426470370955 1.8360667663444457
honey 1.34213194442273 1.3526032442900253
ice water 1

In [115]:
similarity_graphs = []

SUB_THRESHOLD = 0.65

for i in range(len(communities)):
    m = MATRICES[i]
    g = nx.Graph()
    similarity_graphs.append(g)
    
    for ingredient, neighbors in m.items():
        for neighbor, cosine in neighbors.items():
            if cosine >= SUB_THRESHOLD:
                g.add_edge(ingredient, neighbor, cosine=cosine)

In [116]:
graph = similarity_graphs[1]

# Styling
for node, attrs in graph.nodes(data=True):
    attrs['size'] = min(12, max(2, OCCURRENCES[node] / len(OCCURRENCES) * 6))
    # attrs['color'] = communities_colors.get(partition[node], '#BBB')

# Display
Sigma(graph)

Sigma(data={'nodes': [('sage leaves', {'size': 2}), ('pancetta', {'size': 2}), ('chopped fresh sage', {'size':…